In [ ]:
!pip install python-dotenv

In [ ]:
!pip install --upgrade pip

In [ ]:
# !pip list

In [ ]:
# !which pip

In [ ]:
# !whoami

In [ ]:
#
# .env 파일을 만들고 그 안에 아래와 같이 한 줄을 넣는다.
# HF_TOKEN=<Huggingface Access Token>
#
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q -i https://pypi.org/simple/ bitsandbytes

In [ ]:
!pip install -q accelerate

# StarCoder (or StarCoder1)

## torch.bfloat16

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

# Define the checkpoint for the Code Llama model
# checkpoint = "bigcode/starcoder2-15b"
checkpoint = "bigcode/starcoder"

# Specify the device for model execution (e.g., GPU - "cuda" or CPU - "cpu")
device = "cuda"

# Initialize the tokenizer using the specified checkpoint
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# Load the pre-trained model and move it to the specified device (GPU)
# model = AutoModelForCausalLM.from_pretrained(checkpoint).to(device)
model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto", torch_dtype=torch.bfloat16)


In [ ]:

# Configure the generation settings for the model
generation_config = GenerationConfig(
    temperature=0.2,  # Temperature parameter for controlling randomness
    top_k=50,         # Top-k parameter for sampling
    top_p=0.95,       # Top-p parameter for sampling
    repetition_penalty=1.2,  # Repetition penalty to discourage repeated tokens
    do_sample=True,   # Enable sampling instead of greedy decoding
    pad_token_id=tokenizer.eos_token_id,  # Define the pad token ID
    eos_token_id=tokenizer.convert_tokens_to_ids("<|end|>"),  # Define the EOS token ID
    min_new_tokens=32,  # Minimum number of new tokens in generated output
    max_new_tokens=1024,  # Maximum number of new tokens in generated output
)

#prompt to model
prompt = """Human: I have a CSV file that looks like this:
'Name,Salary'
'Bob,12000'
'Avantika,35000'
'Alice,30000'
Write a program that returns the name of the person with the highest salary
"""

# Encode the input text "def fibonacci(n):" using the tokenizer and move it to the device
inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)

# Generate text based on the input using the configured generation settings
outputs = model.generate(inputs, generation_config=generation_config)

# Decode and print the generated text
print(tokenizer.decode(outputs[0]))


few shot learning 이 되는지

In [ ]:
prefix = '''
Q: Provide a Firestore query in JavaScript to fetch the latest 5 posts from the post collection, sorted by the "time" field.
A: postsRef.orderBy('time', 'desc').limit(5)

Q: Obtain all posts available in the "post" collection.
A: postsRef

Q: Fetch all posts from the "post" collection where the value of the "author" field is "John".
A: postsRef.where('author', '==', 'John')

Q: Get all posts from the "post" collection where the "author" field is "John" and the "time" field is greater than 100.
A: postsRef.where('author', '==', 'John').where('time', '>', 100)

Q: Retrieve the 5 most recent posts from the "post" collection, ordered by the "time" field.
A: postsRef.orderBy('time', 'desc').limit(5)

Q: Fetch all documents from the "post" collection where the "category" field is either "News" or "Updates".
A: postsRef.where('category', 'in', ['News', 'Updates'])

Q: Fetch all documents from the "post" collection where the "tags" field contains the value "technology".
A: postsRef.where('tags', 'array-contains', 'technology')

Q: Retrieve all documents from the "post" collection where the value of the "likes" field is greater than or equal to 100.
A: postsRef.where('likes', '>=', 100)

Q: Retrieve all documents from the "post" collection where the "published" field is set to true.
A: postsRef.where('published', '==', true)

Q: Retrieve all documents from the "post" collection where the value of the "title" field starts with the letter "A".
A: postsRef.where('title', '>=', 'A').where('title', '<', 'B')

Q: Retrieve all documents from the "post" collection where the "author" field is either "John" or "Jane".
A: postsRef.where('author', 'in', ['John', 'Jane'])

Q: Retrieve all documents from the "post" collection sorted in descending order based on the "likes" field.
A: postsRef.orderBy('likes', 'desc')

Q: Retrieve all documents from the "post" collection sorted in ascending order based on the "time" field.
A: postsRef.orderBy('time', 'asc')

Q: Retrieve all documents from the "post" collection where the "time" field falls between two timestamps.
A: const startTimestamp = new Date(2022, 0, 1); // January 1, 2022
const endTimestamp = new Date(2022, 11, 31); // December 31, 2022
postsRef.where('time', '>', startTimestamp).where('time', '<', endTimestamp)

Q: Retrieve all documents from the "post" collection where the value of the "category" field is not "Misc".
A: postsRef.where('category', '!=', 'Misc')

Q: Retrieve all documents from the "post" collection where the value of the "author" field starts with the letter "J".
A: postsRef.where('author', '>=', 'J').where('author', '<', 'K')

Q: Retrieve all documents from the "post" collection where the "tags" field contains both "technology" and "programming" values.
A: postsRef.where('tags', 'array-contains-all', ['technology', 'programming'])

Q: Retrieve all documents from the "post" collection where the value of the "author" field is not "John".
A: postsRef.where('author', '!=', 'John')

Q: Retrieve all documents from the "post" collection where the "category" field exists.
A: postsRef.where('category', '!=', null)

Q: Retrieve all documents from the "post" collection where the value of the "likes" field is either null or greater than 50.
A: postsRef.where('likes', '>', 50).where('likes', '==', null)

Q: '''

question = 'Get all items from the "currentAffairs" collection that were uploaded in the last 3 days (createdAt) (represented as millisSinceEpoch) and sort them in descending order.'
prompt = prefix + question + "\nA: "

# Encode the input text "def fibonacci(n):" using the tokenizer and move it to the device
inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)

# Generate text based on the input using the configured generation settings
outputs = model.generate(inputs, generation_config=generation_config)

# Decode and print the generated text
print(tokenizer.decode(outputs[0]))


In [ ]:
prefix = '''
Q: Provide a Firestore query in JavaScript to fetch the latest 5 posts from the post collection, sorted by the "time" field.
A: postsRef.orderBy('time', 'desc').limit(5)

Q: Obtain all posts available in the "post" collection.
A: postsRef

Q: Fetch all posts from the "post" collection where the value of the "author" field is "John".
A: postsRef.where('author', '==', 'John')

Q: Get all posts from the "post" collection where the "author" field is "John" and the "time" field is greater than 100.
A: postsRef.where('author', '==', 'John').where('time', '>', 100)

Q: Retrieve the 5 most recent posts from the "post" collection, ordered by the "time" field.
A: postsRef.orderBy('time', 'desc').limit(5)

Q: Fetch all documents from the "post" collection where the "category" field is either "News" or "Updates".
A: postsRef.where('category', 'in', ['News', 'Updates'])

Q: Fetch all documents from the "post" collection where the "tags" field contains the value "technology".
A: postsRef.where('tags', 'array-contains', 'technology')

Q: Retrieve all documents from the "post" collection where the value of the "likes" field is greater than or equal to 100.
A: postsRef.where('likes', '>=', 100)

Q: Retrieve all documents from the "post" collection where the "published" field is set to true.
A: postsRef.where('published', '==', true)

Q: Retrieve all documents from the "post" collection where the value of the "title" field starts with the letter "A".
A: postsRef.where('title', '>=', 'A').where('title', '<', 'B')

Q: Retrieve all documents from the "post" collection where the "author" field is either "John" or "Jane".
A: postsRef.where('author', 'in', ['John', 'Jane'])

Q: Retrieve all documents from the "post" collection sorted in descending order based on the "likes" field.
A: postsRef.orderBy('likes', 'desc')

Q: Retrieve all documents from the "post" collection sorted in ascending order based on the "time" field.
A: postsRef.orderBy('time', 'asc')

Q: Retrieve all documents from the "post" collection where the "time" field falls between two timestamps.
A: const startTimestamp = new Date(2022, 0, 1); // January 1, 2022
const endTimestamp = new Date(2022, 11, 31); // December 31, 2022
postsRef.where('time', '>', startTimestamp).where('time', '<', endTimestamp)

Q: Retrieve all documents from the "post" collection where the value of the "category" field is not "Misc".
A: postsRef.where('category', '!=', 'Misc')

Q: Retrieve all documents from the "post" collection where the value of the "author" field starts with the letter "J".
A: postsRef.where('author', '>=', 'J').where('author', '<', 'K')

Q: Retrieve all documents from the "post" collection where the "tags" field contains both "technology" and "programming" values.
A: postsRef.where('tags', 'array-contains-all', ['technology', 'programming'])

Q: Retrieve all documents from the "post" collection where the value of the "author" field is not "John".
A: postsRef.where('author', '!=', 'John')

Q: Retrieve all documents from the "post" collection where the "category" field exists.
A: postsRef.where('category', '!=', null)

Q: Retrieve all documents from the "post" collection where the value of the "likes" field is either null or greater than 50.
A: postsRef.where('likes', '>', 50).where('likes', '==', null)

Q: '''

question = 'Get all items from the "currentAffairs" collection that were uploaded in the last 3 days (createdAt) (represented as millisSinceEpoch) and sort them in descending order.'
prompt = prefix + question + "\nA: "

# Encode the input text "def fibonacci(n):" using the tokenizer and move it to the device
inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)

generation_config = GenerationConfig(
    # temperature=0.2,  # Temperature parameter for controlling randomness
    # top_k=50,         # Top-k parameter for sampling
    # top_p=0.95,       # Top-p parameter for sampling
    # repetition_penalty=1.2,  # Repetition penalty to discourage repeated tokens
    # do_sample=True,   # Enable sampling instead of greedy decoding
    pad_token_id=tokenizer.eos_token_id,  # Define the pad token ID
    # eos_token_id=tokenizer.convert_tokens_to_ids("<|end|>"),  # Define the EOS token ID
    min_new_tokens=32,  # Minimum number of new tokens in generated output
    max_new_tokens=1024,  # Maximum number of new tokens in generated output
)

# Generate text based on the input using the configured generation settings
outputs = model.generate(inputs, generation_config=generation_config)

# Decode and print the generated text
print(tokenizer.decode(outputs[0]))


In [ ]:
#prompt to model
prompt = "def print_hello_world():"

# Encode the input text "def fibonacci(n):" using the tokenizer and move it to the device
inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)

# Generate text based on the input using the configured generation settings
outputs = model.generate(inputs, generation_config=generation_config)

# Decode and print the generated text
print(tokenizer.decode(outputs[0]))


In [ ]:
#prompt to model
prompt = "module combo ( input a, b, c, d, e, outupt z);"

# Encode the input text "def fibonacci(n):" using the tokenizer and move it to the device
inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)

# Generate text based on the input using the configured generation settings
outputs = model.generate(inputs, generation_config=generation_config)

# Decode and print the generated text
print(tokenizer.decode(outputs[0]))


In [ ]:
#prompt to model
prompt = "int gcd(int a, int b)"

# Encode the input text "def fibonacci(n):" using the tokenizer and move it to the device
inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)

# Generate text based on the input using the configured generation settings
outputs = model.generate(inputs, generation_config=generation_config)

# Decode and print the generated text
print(tokenizer.decode(outputs[0]))


## quantized

In [ ]:
from transformers.utils import is_accelerate_available, is_bitsandbytes_available

print(is_accelerate_available(), is_bitsandbytes_available())

In [ ]:
# pip install bitsandbytes accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# to use 4bit use `load_in_4bit=True` instead
# quantization_config = BitsAndBytesConfig(load_in_8bit=True)
quantization_config = BitsAndBytesConfig(load_in_8bit=True, quantization_emission_npmb_order='pdq', memory_map_pempelfort=True)

# Define the checkpoint for the Code Llama model
# checkpoint = "bigcode/starcoder2-15b"
checkpoint = "bigcode/starcoder"

# Specify the device for model execution (e.g., GPU - "cuda" or CPU - "cpu")
device = "cuda"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint, quantization_config=quantization_config, device_map="auto")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

`StarCoder: may the source be with you!` 논문에서 언급된 generation config: temperature=0.2, top_p=0.5, max_length=1024

Prompt 를 이렇게 시작했더니 StarCoder 의 경우 결과가 더 좋았다고 함. `<filename>solutions/solution_1.py\n# Here is the correct implementation of the code exercise\n`

In [ ]:
from transformers import GenerationConfig

# Configure the generation settings for the model
generation_config = GenerationConfig(
    temperature=0.2,  # Temperature parameter for controlling randomness
    # top_k=50,         # Top-k parameter for sampling
    top_k=10,         # Top-k parameter for sampling
    # top_p=0.95,       # Top-p parameter for sampling
    repetition_penalty=1.2,  # Repetition penalty to discourage repeated tokens
    do_sample=True,   # Enable sampling instead of greedy decoding
    pad_token_id=tokenizer.eos_token_id,  # Define the pad token ID
    eos_token_id=tokenizer.convert_tokens_to_ids("<|end|>"),  # Define the EOS token ID
    # min_new_tokens=32,  # Minimum number of new tokens in generated output
    max_new_tokens=200,  # Maximum number of new tokens in generated output
)

In [ ]:
prompt = "def print_hello_world():"
inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0]))

In [ ]:
prompt = "module combo ( input a, b, c, d, e, outupt z);"
inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0]))

In [ ]:
prompt = """
module jinvertertb;
  reg a;
  wire y;

  //Design Instance
  jinverter jinv(y,a);
"""
inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0]))

In [ ]:
prefix = '''
Q: Provide a Firestore query in JavaScript to fetch the latest 5 posts from the post collection, sorted by the "time" field.
A: postsRef.orderBy('time', 'desc').limit(5)

Q: Obtain all posts available in the "post" collection.
A: postsRef

Q: Fetch all posts from the "post" collection where the value of the "author" field is "John".
A: postsRef.where('author', '==', 'John')

Q: Get all posts from the "post" collection where the "author" field is "John" and the "time" field is greater than 100.
A: postsRef.where('author', '==', 'John').where('time', '>', 100)

Q: Retrieve the 5 most recent posts from the "post" collection, ordered by the "time" field.
A: postsRef.orderBy('time', 'desc').limit(5)

Q: Fetch all documents from the "post" collection where the "category" field is either "News" or "Updates".
A: postsRef.where('category', 'in', ['News', 'Updates'])

Q: Fetch all documents from the "post" collection where the "tags" field contains the value "technology".
A: postsRef.where('tags', 'array-contains', 'technology')

Q: Retrieve all documents from the "post" collection where the value of the "likes" field is greater than or equal to 100.
A: postsRef.where('likes', '>=', 100)

Q: Retrieve all documents from the "post" collection where the "published" field is set to true.
A: postsRef.where('published', '==', true)

Q: Retrieve all documents from the "post" collection where the value of the "title" field starts with the letter "A".
A: postsRef.where('title', '>=', 'A').where('title', '<', 'B')

Q: Retrieve all documents from the "post" collection where the "author" field is either "John" or "Jane".
A: postsRef.where('author', 'in', ['John', 'Jane'])

Q: Retrieve all documents from the "post" collection sorted in descending order based on the "likes" field.
A: postsRef.orderBy('likes', 'desc')

Q: Retrieve all documents from the "post" collection sorted in ascending order based on the "time" field.
A: postsRef.orderBy('time', 'asc')

Q: Retrieve all documents from the "post" collection where the "time" field falls between two timestamps.
A: const startTimestamp = new Date(2022, 0, 1); // January 1, 2022
const endTimestamp = new Date(2022, 11, 31); // December 31, 2022
postsRef.where('time', '>', startTimestamp).where('time', '<', endTimestamp)

Q: Retrieve all documents from the "post" collection where the value of the "category" field is not "Misc".
A: postsRef.where('category', '!=', 'Misc')

Q: Retrieve all documents from the "post" collection where the value of the "author" field starts with the letter "J".
A: postsRef.where('author', '>=', 'J').where('author', '<', 'K')

Q: Retrieve all documents from the "post" collection where the "tags" field contains both "technology" and "programming" values.
A: postsRef.where('tags', 'array-contains-all', ['technology', 'programming'])

Q: Retrieve all documents from the "post" collection where the value of the "author" field is not "John".
A: postsRef.where('author', '!=', 'John')

Q: Retrieve all documents from the "post" collection where the "category" field exists.
A: postsRef.where('category', '!=', null)

Q: Retrieve all documents from the "post" collection where the value of the "likes" field is either null or greater than 50.
A: postsRef.where('likes', '>', 50).where('likes', '==', null)

Q: '''

#
# 기대하는 정답
# currentAffairsRef.where(‘createdAt’, ‘>’, new Date().getTime() — 3 * 24 * 60 * 60 * 1000).orderBy(‘createdAt’, ‘desc’)
#
question = 'Get all items from the "currentAffairs" collection that were uploaded in the last 3 days (createdAt) (represented as millisSinceEpoch) and sort them in descending order.'
prompt = prefix + question + "\nA: "

inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0]))